In [1]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Lambda
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from matplotlib import cm
import cv2 as cv
from zipfile import ZipFile
import pandas as pd
import os
import numpy as np
from torchvision.io import read_image
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
from torchmetrics import F1Score
from itertools import permutations
from torch.autograd import Variable


import torch.nn as nn 
import torch.nn.functional as F 

trained_weight = False

In [2]:
train_x = []
train_y = []
test_x = []
test_y = []

# Testing Dataset
for idx, i in enumerate(os.listdir("test/good_seed/")):
    view1 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view2 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view3 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view4 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view5 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

    x = [view1, view2, view3, view4, view5]
    y = 1 # Class Good Seed 
    test_x.append(x)
    test_y.append(y)

for idx, i in enumerate(os.listdir("test/bad_seed/")):
    view1 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view2 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view3 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view4 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view5 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

    x = [view1, view2, view3, view4, view5]
    y = 0 # Class Bad Seed 
    test_x.append(x)
    test_y.append(y)

test_x = np.array(test_x)
test_y = np.array(test_y)
test_tensors_x = torch.Tensor(test_x)
test_tensors_y = torch.Tensor(test_y)

test_dataset = TensorDataset(test_tensors_x, test_tensors_y)


# Training Dataset
for idx, i in enumerate(os.listdir("train/good_seed/")):
    view1 = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view2 = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view3 = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view4 = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view5 = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

    x = [view1, view2, view3, view4, view5]
    y = 1 # Class Bad Seed 
    train_x.append(x)
    train_y.append(y)

for idx, i in enumerate(os.listdir("train/bad_seed/")):
    view1 = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view2 = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view3 = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view4 = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
    view5 = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

    x = [view1, view2, view3, view4, view5]
    y = 0 # Class Bad Seed 
    train_x.append(x)
    train_y.append(y)

train_x = np.array(train_x)
train_y = np.array(train_y)
train_tensors_x = torch.Tensor(train_x)
train_tensors_y = torch.Tensor(train_y)

train_dataset = TensorDataset(train_tensors_x, train_tensors_y)


# ( Bad Seed = 78 + Good Seed 81 ) * Amount of Views Permutation 120 = 19080
print('training set', len(train_dataset))

# ( Bad Seed = 28 + Good Seed 18 ) * Amount of Views Permutation 120 = 5520
print('test set', len(test_dataset))



training set 159
test set 46


In [3]:
# test_x = []
# test_y = []

# # Testing Dataset
# for idx, i in enumerate(os.listdir("test/good_seed/")):
#     view1 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view2 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view3 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view4 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view5 = cv.cvtColor(cv.resize(cv.imread('test/good_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

#     x = [view1, view2, view3, view4, view5]
#     y = 1 # Class Good Seed 
#     test_x.append(x)
#     test_y.append(y)

# for idx, i in enumerate(os.listdir("test/bad_seed/")):
#     view1 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x1.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view2 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x2.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view3 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x3.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view4 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x4.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)
#     view5 = cv.cvtColor(cv.resize(cv.imread('test/bad_seed/%s/%s_x5.jpg' % (i,i)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

#     x = [view1, view2, view3, view4, view5]
#     y = 0 # Class Bad Seed 
#     test_x.append(x)
#     test_y.append(y)

# test_x = np.array(test_x)
# test_y = np.array(test_y)
# test_tensors_x = torch.Tensor(test_x)
# test_tensors_y = torch.Tensor(test_y)

# test_dataset = TensorDataset(test_tensors_x, test_tensors_y)



# def gen_train_dataset(view_num):
#     train_x = []
#     train_y = []

#     for idx, i in enumerate(os.listdir("train/good_seed/")):
#         view = cv.cvtColor(cv.resize(cv.imread('train/good_seed/%s/%s_x%s.jpg' % (i,i,view_num)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

#         x = view
#         y = 1 # Class Good Seed 
#         train_x.append(x)
#         train_y.append(y)

#     for idx, i in enumerate(os.listdir("train/bad_seed/")):
#         view = cv.cvtColor(cv.resize(cv.imread('train/bad_seed/%s/%s_x%s.jpg' % (i,i, view_num)), (64,64), interpolation = cv.INTER_AREA), cv.COLOR_BGR2RGB)

#         x = view
#         y = 0 # Class Bad Seed 
#         train_x.append(x)
#         train_y.append(y)
        

#     train_x = np.array(train_x)
#     train_y = np.array(train_y)
#     train_tensors_x = torch.Tensor(train_x)
#     train_tensors_y = torch.Tensor(train_y)
#     train_dataset = TensorDataset(train_tensors_x, train_tensors_y)

#     return train_dataset

# train_dataset_view1 = gen_train_dataset(1)
# train_dataset_view2 = gen_train_dataset(2)
# train_dataset_view3 = gen_train_dataset(3)
# train_dataset_view4 = gen_train_dataset(4)
# train_dataset_view5 = gen_train_dataset(5)

# # print('Training Set (View1) : ', len(train_dataset_view1))
# # print('Training Set (View2) : ', len(train_dataset_view2))
# # print('Training Set (View3) : ', len(train_dataset_view3))
# # print('Training Set (View4) : ', len(train_dataset_view4))
# # print('Training Set (View5) : ', len(train_dataset_view5))
# # print('Testing Set : ', len(test_dataset))

In [4]:
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=2)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

In [5]:
class FeatureNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
                    nn.Conv2d(3, 32, kernel_size=3, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
                    nn.BatchNorm2d(64),

                    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
                    nn.BatchNorm2d(128),

                    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
                    nn.ReLU(),
                    nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
                    nn.BatchNorm2d(256))

    def forward(self, xb):
        return self.network(xb)


class ClassificationNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
                    nn.Linear(81920, 1024),
                    nn.ReLU(),
                    nn.Linear(1024, 2))

    def forward(self, xb):
        return self.network(xb)



f1_net = FeatureNet()
f2_net = FeatureNet()
f3_net = FeatureNet()
f4_net = FeatureNet()
f5_net = FeatureNet()

model = ClassificationNet()

In [6]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.network = nn.Sequential(
#                     nn.Conv2d(5, 32, kernel_size=3, padding=1),
#                     nn.ReLU(),
#                     nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
#                     nn.ReLU(),
#                     nn.MaxPool2d(2, 2), # output: 64 x 16 x 16
#                     nn.BatchNorm2d(64),

#                     nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
#                     nn.ReLU(),
#                     nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
#                     nn.ReLU(),
#                     nn.MaxPool2d(2, 2), # output: 128 x 8 x 8
#                     nn.BatchNorm2d(128),

#                     nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
#                     nn.ReLU(),
#                     nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
#                     nn.ReLU(),
#                     nn.MaxPool2d(2, 2), # output: 256 x 4 x 4
#                     nn.BatchNorm2d(256),

#                     nn.Flatten(), 
#                     nn.Linear(16384, 1024),
#                     nn.ReLU(),
#                     nn.Linear(1024, 2))

#     def forward(self, xb):
#         return self.network(xb)

# model = Net()

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using {} device'.format(device))
model.to(device)
f1_net.to(device)
f2_net.to(device)
f3_net.to(device)
f4_net.to(device)
f5_net.to(device)

Using cuda:0 device


FeatureNet(
  (network): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [8]:
def initialize_weights(model):
  if isinstance(model, nn.Conv2d):
      nn.init.kaiming_uniform_(model.weight.data,nonlinearity='relu')
      if model.bias is not None:
          nn.init.constant_(model.bias.data, 0)
  elif isinstance(model, nn.BatchNorm2d):
      nn.init.constant_(model.weight.data, 1)
      nn.init.constant_(model.bias.data, 0)
  elif isinstance(model, nn.Linear):
      nn.init.kaiming_uniform_(model.weight.data)
      nn.init.constant_(model.bias.data, 0)


In [9]:
model.apply(initialize_weights)
f1_net.apply(initialize_weights)
f2_net.apply(initialize_weights)
f3_net.apply(initialize_weights)
f4_net.apply(initialize_weights)
f5_net.apply(initialize_weights)

if trained_weight:
  checkpoint = torch.load('grayscale.pt')
  model.load_state_dict(checkpoint['model_state_dict'])
  print('Weights Loaded')

In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_main = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer_f1 = optim.SGD(f1_net.parameters(), lr=0.001, momentum=0.9)
optimizer_f2 = optim.SGD(f2_net.parameters(), lr=0.001, momentum=0.9)
optimizer_f3 = optim.SGD(f3_net.parameters(), lr=0.001, momentum=0.9)
optimizer_f4 = optim.SGD(f4_net.parameters(), lr=0.001, momentum=0.9)
optimizer_f5 = optim.SGD(f5_net.parameters(), lr=0.001, momentum=0.9)

In [11]:
# features_f1 = []
# for images, labels in train_dataloader_view1:
#         images = images.view(images.shape[0], images.shape[3], images.shape[1], images.shape[2])
#         feature = f1_net(images.cuda())
#         features_f1.append(feature)

# features_f2 = []
# for images, labels in train_dataloader_view2:
#         images = images.view(images.shape[0], images.shape[3], images.shape[1], images.shape[2])
#         feature = f2_net(images.cuda())
#         features_f2.append(feature)

# features_f3 = []
# for images, labels in train_dataloader_view3:
#         images = images.view(images.shape[0], images.shape[3], images.shape[1], images.shape[2])
#         feature = f3_net(images.cuda())
#         features_f3.append(feature)

# features_f4 = []
# for images, labels in train_dataloader_view4:
#         images = images.view(images.shape[0], images.shape[3], images.shape[1], images.shape[2])
#         feature = f4_net(images.cuda())
#         features_f4.append(feature)

# features_f5 = []
# for images, labels in train_dataloader_view5:
#         images = images.view(images.shape[0], images.shape[3], images.shape[1], images.shape[2])
#         feature = f5_net(images.cuda())
#         features_f5.append(feature)

In [12]:
from torch.autograd import Variable

if trained_weight == False:

  # Training with Validation 
  n_epochs = 100 
  last25 = 0
  data_loaders = {'train': train_dataloader}
  data_lengths = {'train': len(train_dataset)}
  feature_map_shape = [8, 256, 8, 8]

  for epoch in range(n_epochs):
      print('Epoch {}/{}'.format(epoch, n_epochs - 1))
      print('-' * 10)
      model.train(True)  
      running_loss = 0.0

          # Iterate over data.
      for images, labels in train_dataloader:
              # wrap them in a torch Variable
              images, labels = Variable(images), Variable(labels)

              # torch.Size([8, 5, 64, 64, 3]) --> torch.Size([5, 8, 3, 64, 64])
              images = images.view(images.shape[1], images.shape[0], images.shape[4], images.shape[2],images.shape[3])

              # convert variables to floats for regression loss
              labels = labels.type(torch.LongTensor).to(device)
              images = images.type(torch.FloatTensor).to(device)

              feature_map_1 = torch.empty(feature_map_shape)
              feature_map_2 = torch.empty(feature_map_shape)
              feature_map_3 = torch.empty(feature_map_shape)
              feature_map_4 = torch.empty(feature_map_shape)
              feature_map_5 = torch.empty(feature_map_shape)

              view_num = 0

              for seed_view in images:
                view_num += 1
                if view_num == 1:
                  # ([8, 256, 8, 8]), 8 = Batch, (256, 8, 8) = Feature Map
                  feature_map_1 = f1_net(seed_view)
                elif view_num == 2:
                  feature_map_2 = f2_net(seed_view)
                elif view_num == 3:
                  feature_map_3 = f3_net(seed_view)
                elif view_num == 4:
                  feature_map_4 = f4_net(seed_view)
                elif view_num == 5:
                  feature_map_5 = f5_net(seed_view)

              temp = []
              temp2 = []

              for i in range(feature_map_1.shape[0]):
                temp.append(torch.cat((feature_map_1[i], feature_map_2[i], feature_map_3[i], feature_map_4[i], feature_map_5[i]), 0))

              for j in temp:
                temp2.append(j.flatten())
              
              # torch.Size([8, 81920])
              feature_map_concat = torch.stack(temp2)

              # forward pass to get outputs
              output_labels = model(feature_map_concat)

              print(output_labels.shape)
              print(labels.shape)

              # calculate the loss between predicted and target keypoints
              loss = criterion(output_labels, labels)

              # zero the parameter (weight) gradients
              optimizer_main.zero_grad()
              optimizer_f1.zero_grad()
              optimizer_f2.zero_grad()
              optimizer_f3.zero_grad()
              optimizer_f4.zero_grad()
              optimizer_f5.zero_grad()

              # backward + optimize only if in training phase
              loss.backward()
              # update the weights
              optimizer_main.step()
              optimizer_f1.step()
              optimizer_f2.step()
              optimizer_f3.step()
              optimizer_f4.step()
              optimizer_f5.step()

              # print loss statistics
              running_loss += loss.item()

      epoch_loss = running_loss / len(train_dataset)
      last25 += epoch_loss
      print('Training Loss: {:.8f}'.format(epoch_loss))

      if epoch >= 25 and epoch % 25 == 0:
        last25avg = last25 / 25
        last25 = 0
        print('# Last 25 AVG: {:.8f} #'.format(last25avg))

# torch.save({
#             'model_state_dict': model.state_dict(),
#             }, 'grayscale_self.model.pt')

Epoch 0/2
----------
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([7, 256, 8, 8])
0
1
2
3
4
5
6
Training Loss: 0.49814553
Epoch 1/2
----------
torch.Size([8, 256, 8, 8])
0
1
2
3
4
5
6
7
torch.Size([8, 256, 8, 8])
0
1


In [13]:
model.eval()

total = 0  # keeps track of how many images we have processed 
correct = 0 # keeps track of how many correct images our self.model predicts
true_pos_and_neg = 0
true_pos = 0
true_neg = 0
false_pos = 0
false_neg = 0

with torch.no_grad():
      for images, labels in test_dataloader:

        # wrap them in a torch Variable
        images, labels = Variable(images), Variable(labels)

        # convert variables to floats for regression loss
        labels = labels.type(torch.LongTensor).to(device)
        images = images.type(torch.FloatTensor).to(device)

        

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size()[0]

        true_pos_and_neg += sum(map(lambda x,y : x == y, labels, predicted)) 
        true_pos += sum(map(lambda x,y : x == y and x == 1, labels, predicted))
        true_neg += sum(map(lambda x,y : x == y and x == 0, labels, predicted))
        false_pos += sum(map(lambda x,y : x != y and x == 0, labels, predicted))
        false_neg += sum(map(lambda x,y : x != y and x == 1, labels, predicted)) 
        correct += (predicted == labels).sum().item()

precision = true_pos / (true_pos + false_pos)
recall = true_pos / (true_pos + false_neg)
f1 = 2 * ((precision * recall) / (precision + recall))
        
print('\n==============Results==============')
print("Accuracy: ", correct/total)
print("F1 score: ", f1.cpu().numpy())
print('\n')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (163840x3 and 81920x1024)